<a href="https://colab.research.google.com/github/Dario66/Deep-Learning/blob/main/Fruit360Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.datasets import fashion_mnist # carica il Fashion-MNIST dataset
import tensorflow as tf
from torchsummary import summary
from sklearn.model_selection import KFold
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.layers import LSTM,Dense, Flatten
import matplotlib.pyplot as plt

(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
# summarize loaded dataset
print('Train: X=%s, Y=%s' % (trainX.shape, trainY.shape))
print('Test: X=%s, Y=%s' % (testX.shape, testY.shape))

# load train and test dataset
def load_dataset():
    # load dataset
    ((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    print (len(trainX))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    print (len(testX))
    # one hot encode target values
    trainY = tf.keras.utils.to_categorical(trainY)
    testY = tf.keras.utils.to_categorical(testY)
    return (trainX, trainY, testX, testY)
 
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')

	singleArray1=train_norm[0][0]

	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0

	singleArray1=train_norm[0][0]

	# return normalized images
	return train_norm, test_norm
 
# define cnn model
def define_model():
	model = tf.keras.Sequential()
	model.add(Conv2D(32, (5, 5), padding='same',activation='relu', kernel_initializer='he_uniform',input_shape=(28, 28, 1)))
	#numero di parametri 640 --> 3^2*1*64=576+64
	#output del layer Conv--->(28-3+2*0/1)+1=26x26x64
	model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))
	model.add(Conv2D(64, (5, 5),padding='same', activation='relu', kernel_initializer='he_uniform'))
	model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	#model.summary()
	# compile model
	opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# define cnn model
def define_model2():
	model2 = tf.keras.Sequential()
	model2.add(Conv2D(32,(5,5), strides=(1,1),input_shape=(28,28,1),padding='same',activation='relu'))
	model2.add(tf.keras.layers.Activation("relu"))
	model2.add(tf.keras.layers.MaxPooling2D((2,2)))
	model2.add(Conv2D(64,(5,5), strides=(1,1), padding='same' ))
	model2.add(tf.keras.layers.Activation("relu"))
	model2.add(tf.keras.layers.MaxPooling2D((2,2)))
	model2.add(Flatten())
	model2.add(Dense(1024))
	model2.add(tf.keras.layers.Activation("relu"))
	model2.add(tf.keras.layers.Dropout(0.5))
	model2.add(Dense(10, activation='softmax'))
	model2.add(tf.keras.layers.Activation("relu"))
	return model2


# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model2 = define_model2()
		# select rows for train and test
		trainX, trainY, testX, te1stY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model2.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model2.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# append scores
		scores.append(acc)
		histories.append(history)
	return scores, histories
 
# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		plt.subplot(211)
		plt.title('Cross Entropy Loss')
		plt.plot(histories[i].history['loss'], color='blue', label='train')
		plt.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		plt.subplot(212)
		plt.title('Classification Accuracy')
		plt.plot(histories[i].history['accuracy'], color='blue', label='train')
		plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	plt.show()
 
# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, np.std(scores)*100, len(scores)))
	# box and whisker plots of results
	plt.boxplot(scores)
	plt.show()
 
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# evaluate model
	scores, histories = evaluate_model(trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)
# example of loading the fashion mnist dataset

def show_prova():
 (trainX, trainy), (testX, testy) = fashion_mnist.load_data()
 # summarize loaded dataset
 print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
 print('Test: X=%s, y=%s' % (testX.shape, testy.shape))
 # plot first few images
 for i in range(9):
	 # define subplot
	 plt.subplot(330 + 1 + i)
	 # plot raw pixel data
	 plt.imshow(trainX[i], cmap=plt.get_cmap('gray'))
 # show the figure
 plt.show() 

# entry point, run the test harness
#show_prova()
run_test_harness()

In [ ]:
# evaluate the deep model on the test dataset
from keras.datasets import fashion_mnist
from keras.models import load_model

 
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = tf.keras.utils.to_categorical(trainY)
	testY = tf.keras.utils.to_categorical(testY)
	return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm
 
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# load model
	model = load_model('final_model.h5')
	# evaluate model on test dataset
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
 
# entry point, run the test harness
run_test_harness()

> 91.580


In [ ]:
# save the final model to file
from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.datasets import fashion_mnist # carica il Fashion-MNIST dataset
import tensorflow as tf
from torchsummary import summary
from sklearn.model_selection import KFold
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.layers import LSTM,Dense, Flatten
import matplotlib.pyplot as plt
 
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = fashion_mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = tf.keras.utils.to_categorical(trainY)
	testY = tf.keras.utils.to_categorical(testY)
	return trainX, trainY, testX, testY
 
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm
 
# define cnn model
def define_model():
	model = tf.keras.Sequential()
	model.add(Conv2D(64, (3, 3), padding='same',activation='relu', kernel_initializer='he_uniform',input_shape=(28, 28, 1)))
	#numero di parametri 640 --> 3^2*1*64=576+64
	#output del layer Conv--->(28-3+2*0/1)+1=26x26x64
	model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3),strides=(2, 2)))
	model.add(Conv2D(64, (3, 3),padding='same', activation='relu', kernel_initializer='he_uniform'))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	#model.summary()
	# compile model
	opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model
 
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
	# save model
	model.save('final_model.h5')
 
# entry point, run the test harness
run_test_harness()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
from keras.layers.normalization.batch_normalization import BatchNormalization
from keras.datasets import fashion_mnist # carica il Fashion-MNIST dataset
import tensorflow as tf
from torchsummary import summary
from sklearn.model_selection import KFold
from statistics import mean
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.layers import LSTM,Dense, Flatten
import matplotlib.pyplot as plt

from PIL import Image
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = tf.keras.utils.load_img(filename, color_mode = "grayscale", target_size=(28, 28))
	# convert to array
	img = tf.keras.utils.img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('uno.png')
	# load model
	model = tf.keras.models.load_model('final_model.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
 
# entry point, run the example
run_example()

In [80]:
! kaggle datasets download -d moltean/fruits

 99% 1.27G/1.28G [00:05<00:00, 191MB/s]
100% 1.28G/1.28G [00:05<00:00, 231MB/s]


In [ ]:
! unzip fruits.zip

In [3]:
from keras.layers.normalization.batch_normalization import BatchNormalization
import tensorflow as tf
from torchsummary import summary
from sklearn.model_selection import KFold
from statistics import mean
import matplotlib.pyplot as plt
from keras.layers import Conv2D
from keras.layers import LSTM,Dense, Flatten
import os
from PIL import Image
import pandas as pd
import random
import seaborn as sns
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from collections import Counter
import numpy as np
from glob import glob
from google.colab import drive
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from sklearn.datasets import load_files
from keras.utils import np_utils
#super useful
#https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html


# In[5]:


def load_dataset(path):
    data=load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files, targets, target_labels

X_train, y_train, target_labels = load_dataset('fruits-360_dataset/fruits-360/Training')

X_test, y_test, _ = load_dataset('fruits-360_dataset/fruits-360/Test')

#numero di classi presenti
className = glob('fruits-360_dataset/fruits-360/Training' + '/*')
n_class = len(className)
print('numero di classi:'+ str(n_class))

y_train=np_utils.to_categorical(y_train, n_class)
y_test=np_utils.to_categorical(y_test, n_class)

#funzione per convertire i dati in array
def convert_to_array(pics):
    img_arr =[]
    for pic in pics:
        img_arr.append(np.asarray(load_img(pic)))
    return img_arr

X_train = np.array(convert_to_array(X_train))
X_test = np.array(convert_to_array(X_test))

#X_train = X_train.astype('float32')/255
#X_test = X_test.astype('float32')/255



numero di classi:131
